In [4]:
import h3
lat, lng = 37.769377, -122.388903
resolution = 9
h3.latlng_to_cell(lat, lng, resolution)

'89283082e73ffff'

In [5]:
h3.versions()


{'c': '4.1.0', 'python': '4.1.2'}

**Cell counts at different resolutions**

In [7]:
from tabulate import tabulate

def num_pentagons(res):
    return 12

def num_hexagons(res):
    'Number of *hexagons* (excluding pentagons) at a resolution'
    return h3.get_num_cells(res) - 12

def num_cells(res):
    'Number of *hexagons* (excluding pentagons) at a resolution'
    return h3.get_num_cells(res)  # function name to be fixed in 4.0 release

def fmt(num):
    s = '{:,.0f}'
    return s.format(num)  


counts = [
    (res, num_cells(res), num_hexagons(res), num_pentagons(res))
    for res in range(16)
]

counts = [
    tuple(map(fmt, row))
    for row in counts
]
counts

[('0', '122', '110', '12'),
 ('1', '842', '830', '12'),
 ('2', '5,882', '5,870', '12'),
 ('3', '41,162', '41,150', '12'),
 ('4', '288,122', '288,110', '12'),
 ('5', '2,016,842', '2,016,830', '12'),
 ('6', '14,117,882', '14,117,870', '12'),
 ('7', '98,825,162', '98,825,150', '12'),
 ('8', '691,776,122', '691,776,110', '12'),
 ('9', '4,842,432,842', '4,842,432,830', '12'),
 ('10', '33,897,029,882', '33,897,029,870', '12'),
 ('11', '237,279,209,162', '237,279,209,150', '12'),
 ('12', '1,660,954,464,122', '1,660,954,464,110', '12'),
 ('13', '11,626,681,248,842', '11,626,681,248,830', '12'),
 ('14', '81,386,768,741,882', '81,386,768,741,870', '12'),
 ('15', '569,707,381,193,162', '569,707,381,193,150', '12')]

In [8]:
headers = [
    'Res',
    'Total number of cells',
    'Number of hexagons',
    'Number of pentagons',
]
out = tabulate(counts, headers=headers, tablefmt='pipe', stralign='right')

print(out)

|   Res |   Total number of cells |   Number of hexagons |   Number of pentagons |
|------:|------------------------:|---------------------:|----------------------:|
|     0 |                     122 |                  110 |                    12 |
|     1 |                     842 |                  830 |                    12 |
|     2 |                   5,882 |                5,870 |                    12 |
|     3 |                  41,162 |               41,150 |                    12 |
|     4 |                 288,122 |              288,110 |                    12 |
|     5 |               2,016,842 |            2,016,830 |                    12 |
|     6 |              14,117,882 |           14,117,870 |                    12 |
|     7 |              98,825,162 |           98,825,150 |                    12 |
|     8 |             691,776,122 |          691,776,110 |                    12 |
|     9 |           4,842,432,842 |        4,842,432,830 |                    12 |
|   

In [9]:
from IPython.display import Markdown
Markdown(out)

|   Res |   Total number of cells |   Number of hexagons |   Number of pentagons |
|------:|------------------------:|---------------------:|----------------------:|
|     0 |                     122 |                  110 |                    12 |
|     1 |                     842 |                  830 |                    12 |
|     2 |                   5,882 |                5,870 |                    12 |
|     3 |                  41,162 |               41,150 |                    12 |
|     4 |                 288,122 |              288,110 |                    12 |
|     5 |               2,016,842 |            2,016,830 |                    12 |
|     6 |              14,117,882 |           14,117,870 |                    12 |
|     7 |              98,825,162 |           98,825,150 |                    12 |
|     8 |             691,776,122 |          691,776,110 |                    12 |
|     9 |           4,842,432,842 |        4,842,432,830 |                    12 |
|    10 |          33,897,029,882 |       33,897,029,870 |                    12 |
|    11 |         237,279,209,162 |      237,279,209,150 |                    12 |
|    12 |       1,660,954,464,122 |    1,660,954,464,110 |                    12 |
|    13 |      11,626,681,248,842 |   11,626,681,248,830 |                    12 |
|    14 |      81,386,768,741,882 |   81,386,768,741,870 |                    12 |
|    15 |     569,707,381,193,162 |  569,707,381,193,150 |                    12 |

In [12]:
from shapely.geometry import Point
import geopandas as gpd
import numpy as np
import random
import swifter

# Define the range for ELAPSETIME
start = 0
stop = 500

# Define the bounding box for Boulder, CO
minx, miny = -105.301758, 39.964069
maxx, maxy = -105.178925, 40.094555

# Generate a list of random lat-long points within the bounding box
points = [Point(random.uniform(minx, maxx), random.uniform(miny, maxy)) for _ in np.arange(10000)]

# Create a geodataframe with the specified columns and attributes
gdf = gpd.GeoDataFrame({'ELAPSETIME': [random.randint(start, stop) for _ in np.arange(10000)],
                        'geometry': points}, crs='EPSG:4326')

gdf.explore(color='orangered', tiles='CartoDB positron')

In [18]:
from shapely.geometry import Polygon

def cell_to_shapely(cell):
    coords = h3.cell_to_boundary(cell)
    flipped = tuple(coord[::-1] for coord in coords)
    return Polygon(flipped)

In [33]:
# x = longitude, y = latitude!!!
res = 7
col = f"H3_{res}"
gdf[col] = gdf.swifter.apply(lambda row: str(h3.latlng_to_cell(row.geometry.y, row.geometry.x, res)), axis=1)
h3_df = gdf.groupby(col)['ELAPSETIME'].describe().reset_index()
h3_df.info()
h3_df.head()

Pandas Apply:   0%|          | 0/10000 [00:00<?, ?it/s]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40 entries, 0 to 39
Data columns (total 9 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   H3_7    40 non-null     object 
 1   count   40 non-null     float64
 2   mean    40 non-null     float64
 3   std     39 non-null     float64
 4   min     40 non-null     float64
 5   25%     40 non-null     float64
 6   50%     40 non-null     float64
 7   75%     40 non-null     float64
 8   max     40 non-null     float64
dtypes: float64(8), object(1)
memory usage: 2.9+ KB


,H3_7,count,mean,std,min,25%,50%,75%,max
0,872681a00ffffff,379.0,241.926121,149.415510,7.0,110.00,239.0,366.0,500.0
1,872681a01ffffff,291.0,244.295533,143.539861,0.0,128.50,233.0,373.5,500.0
2,872681a02ffffff,319.0,257.388715,143.588727,0.0,136.00,257.0,385.0,499.0
3,872681a03ffffff,70.0,230.742857,147.593811,7.0,93.25,224.0,366.0,492.0
4,872681a04ffffff,345.0,251.759420,141.861493,1.0,130.00,258.0,376.0,500.0


In [34]:
h3_geoms = h3_df[col].apply(lambda x: cell_to_shapely(x))
h3_gdf = gpd.GeoDataFrame(data=h3_df, geometry=h3_geoms, crs=4326)
h3_gdf.explore(color='orangered', tiles='CartoDB positron')